In [40]:
import re 
import nltk
import csv
import pandas as pd
import glob
import time
import string
import xlrd
nltk.set_proxy('https://genproxy:8080')
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tanvith\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [41]:
with open('keywords.txt', "r") as pldt_list:
    words = pldt_list.read().split('\n')
#words.remove('')
print(words)

['enterprise', 'customer', 'create', 'encompass', 'router', 'siebel', 'crm', 'odo', 'notification', 'provide', 'lead-to-cash', 'e-wan', 'vpn', 'new', 'creation', 'CSR', 'e-line', 'cancellation', 'reason', 'Clarity', 'service ', 'order', 'partial', 'BRAINS', 'IPVPN', 'full', 'Cease', 'modify', 'removing', 'deactivitation', 'cancel']


In [42]:
#Preparing the training file
with open ("featurefile.csv","w+") as featurefile:
    fields=['MTV No.']+words+['Order Action','Sales Channel']
    writer = csv.DictWriter(featurefile, fieldnames=fields)
    writer.writeheader()
    '''Read the data from excel'''
    FileName='Training set.xlsx'
    df = pd.read_excel(FileName,sheet_name='Sheet1')
    ReadList=df.columns.values.tolist()
    #print(ReadList)
    #print(df['MTV Description'][0:])
    #print(df['Order Action'][0:])
    Sales_channel=df['Sales Channel'][0:].values.tolist()
    Order_Action=df['Order Action'][0:].values.tolist()
    MTV_description=df[['MTV Description']][0:].values.tolist()
    #print(MTV_description)
    #print(Order_Action)
    #print(Sales_channel)
    for i in range(0,len(MTV_description)):
        featurefile.write(str(i+1))
        featurefile.write('\t\t')
        #print(MTV_description[0])
        for desc in MTV_description[i]: 
            desc_to_string = ''.join(desc)
            desc_to_string = desc_to_string.lower() 
            desc_to_string=desc_to_string.replace('.'+','+':',' ')
            #print(desc_to_string)
            desc_words = desc_to_string.split()
            #print(desc_words)            
            for word in words:
                if word in desc_words:
                    featurefile.write('1\t\t')
                else:
                    featurefile.write('0\t\t')           
        featurefile.write(str(Order_Action[i]))
        featurefile.write('\t\t')
        featurefile.write(str(Sales_channel[i]))
        featurefile.write('\n')

In [43]:
#Preparing testing file
with open ("testfile.csv","w+") as testfile:
    fields_test=['MTV No.']+words
    #fields_test = ['MTV No.','enterprise','customer','create','encompass','router','siebel','crm','odo','notification','provide','lead-to-cash','e-wan','vpn','new','creation','csr','e-line','vpn-dsl','dsl-vpn','unplg-s&m','cancellation','reason','clarity','service','order','partial','brains','ipvpn','full','cease','removing','deactivation','Order Action']
    writer_test = csv.DictWriter(testfile, fieldnames=fields_test)
    writer_test.writeheader()
    '''Read the data from excel'''
    FileName_test='Testing set.xlsx'
    df_test = pd.read_excel(FileName_test,sheet_name='Sheet1')
    ReadList_test=df_test.columns.values.tolist()
    #print(ReadList_test)
    #print(df_test['MTV Description'][0:])
    MTV_description_test=df_test[['MTV Description']][0:].values.tolist()
    #print(MTV_description_test[0])
    for i in range(0,len(MTV_description_test)):
        testfile.write(str(i+1))
        testfile.write('\t\t')
        #print(MTV_description[0])
        for desc1 in MTV_description_test[i]: 
            desc1_to_string = ''.join(desc1)
            desc1_to_string = desc1_to_string.lower()
            desc1_to_string=desc1_to_string.replace('.'+','+':',' ')
            #print(desc1_to_string)
            desc_words1 = desc1_to_string.split()
            #print(desc1_words) 
            for word in words:
                if word in desc_words1:
                    testfile.write('1\t\t')
                else:
                    testfile.write('0\t\t')                   
        testfile.write('\n')

In [44]:
#Machine Learning
import numpy as pd
import pandas as pd
from pandas import Series,DataFrame

In [45]:
featurefile_df = pd.read_csv('featurefile.csv',sep=',|\t\t|\n',header=0,engine='python')
featurefile_df.head()

,MTV No.,enterprise,customer,create,encompass,router,siebel,crm,odo,notification,...,BRAINS,IPVPN,full,Cease,modify,removing,deactivitation,cancel,Order Action,Sales Channel
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,Provide,Call center
1,2,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,Provide,Call center
2,3,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,Provide,Call center
3,4,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,Provide,Call center
4,5,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,Provide,Call center


In [46]:
testfile_df = pd.read_csv('testfile.csv',sep= ',|\t\t|\n',header=0,engine='python')
testfile_df.head()

,MTV No.,enterprise,customer,create,encompass,router,siebel,crm,odo,notification,...,order,partial,BRAINS,IPVPN,full,Cease,modify,removing,deactivitation,cancel
0,1,1,1,1,1,0,1,1,1,1,...,1,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,0,1,0,1,1,...,1,0,0,0,0,0,0,0,0,0
2,3,0,0,0,1,0,1,0,1,1,...,1,0,0,0,0,0,0,0,0,1
3,4,0,0,0,1,0,1,0,1,1,...,1,0,0,0,0,0,1,0,0,0
4,5,0,0,0,1,0,1,0,1,1,...,1,1,0,0,1,0,1,0,0,1


In [74]:
featurefile_data = featurefile_df[words]
#featurefile_data = featurefile_df[['enterprise','customer','create','encompass','router','siebel','crm','odo','notification','provide','lead-to-cash','e-wan','vpn','new','creation','csr','e-line','vpn-dsl','dsl-vpn','unplg-s&m','cancellation','reason','clarity','service','order','partial','brains','ipvpn','full','cease','removing','deactivation']]
featurefile_data.head()

,enterprise,customer,create,encompass,router,siebel,crm,odo,notification,provide,...,order,partial,BRAINS,IPVPN,full,Cease,modify,removing,deactivitation,cancel
0,1,1,1,1,1,1,1,1,1,1,...,1,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,1,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,1,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,1,...,1,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,...,1,0,0,0,0,0,0,0,0,0


In [48]:
testfile_data = testfile_df[words]
#testfile_data = testfile_df[['enterprise','customer','create','encompass','router','siebel','crm','odo','notification','provide','lead-to-cash','e-wan','vpn','new','creation','csr','e-line','vpn-dsl','dsl-vpn','unplg-s&m','cancellation','reason','clarity','service','order','partial','brains','ipvpn','full','cease','removing','deactivation']]
testfile_data.head()

,enterprise,customer,create,encompass,router,siebel,crm,odo,notification,provide,...,order,partial,BRAINS,IPVPN,full,Cease,modify,removing,deactivitation,cancel
0,1,1,1,1,0,1,1,1,1,1,...,1,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,1,0,1,1,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,1,0,1,1,0,...,1,0,0,0,0,0,0,0,0,1
3,0,0,0,1,0,1,0,1,1,0,...,1,0,0,0,0,0,1,0,0,0
4,0,0,0,1,0,1,0,1,1,0,...,1,1,0,0,1,0,1,0,0,1


In [49]:
featurefile_target_orderaction = featurefile_df['Order Action']
featurefile_target_orderaction.tail()

42             Cease
43      Cancel cease
44            Modify
45     Cancel Modify
46    Cancel Provide
Name: Order Action, dtype: object

In [50]:
featurefile_target_saleschanel = featurefile_df['Sales Channel']
featurefile_target_saleschanel.head()

0    Call center
1    Call center
2    Call center
3    Call center
4    Call center
Name: Sales Channel, dtype: object

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [52]:
logreg = LogisticRegression()
feature_data_train,feature_data_test,feature_target_orderaction_train,feature_target_orderaction_test = train_test_split(featurefile_data,featurefile_target_orderaction,test_size=0.3,random_state=3)

In [53]:
logreg.fit(feature_data_train,feature_target_orderaction_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [54]:
target_pred_orderaction = logreg.predict(feature_data_test)
print (target_pred_orderaction)

['Cancel Provide' 'Cease' 'Cease' 'Cease' 'Cease' 'Cancel Provide'
 'Cancel Provide' 'Cancel Provide' 'Provide' 'Cease' 'Cancel Provide'
 'Cancel Provide' 'Cease' 'Provide' 'Cancel Provide']


In [55]:
order_action = logreg.predict(testfile_data)
print (order_action)

['Provide' 'Cease' 'Cancel cease' 'Cease' 'Cancel Provide' 'Cease']


In [56]:
print(metrics.accuracy_score(feature_target_orderaction_test,target_pred_orderaction))

0.8666666666666667


In [57]:
sales_channel= ['Call centre','Call centre','Call centre','Call centre','Call centre','Call centre']
print(sales_channel)

['Call centre', 'Call centre', 'Call centre', 'Call centre', 'Call centre', 'Call centre']


In [58]:
with open('product_keywords.txt', "r") as product_list:
    products = product_list.read().split('\n')
#products.remove('')
print(products)

['encompass', 'router', 'metro', 'e-wan', 'vpn', 'e-line', 'brains', 'ipvpn', 'vpn-dsl', 'swup', 'vpn-dsl', 'embed', 'shops.work/vpn/dsl', 'shops.work/unplg-s&m', 'shops.work/unplg-s&p', 'shops.work/unplg-so', 'l2/l3']


In [59]:
#Preparing the product prediction training file
with open ("product_featurefile.csv","w+") as product_featurefile:
    fields_product=['MTV No.']+products+['Product']
    writer_product = csv.DictWriter(product_featurefile, fieldnames=fields_product)
    writer_product.writeheader()
    '''Read the data from excel'''
    FileName='Training set.xlsx'
    df = pd.read_excel(FileName,sheet_name='Sheet1')
    ReadList=df.columns.values.tolist()
    #print(ReadList)
    #print(df['Product'][0:])
    product=df['Product'][0:].values.tolist()
    #print(product)
    for i in range(0,len(MTV_description)):
        product_featurefile.write(str(i+1))
        product_featurefile.write('\t\t')
        #print(MTV_description[0])
        for desc2 in MTV_description[i]: 
            desc2_to_string = ''.join(desc2)
            desc2_to_string = desc2_to_string.lower()           
            #desc2_to_string=desc2_to_string.replace('.',' ')
            desc2_to_string=desc2_to_string.replace(',',' ')
            desc2_to_string=desc2_to_string.replace(':',' ')
            #print(desc_to_string)
            desc2_words = desc2_to_string.split()
            #print(desc_words)            
            for word1 in products:
                if word1 in desc2_words:
                    product_featurefile.write('1\t\t')
                else:
                    product_featurefile.write('0\t\t')
        product_featurefile.write(str(product[i]))
        product_featurefile.write('\n')

In [60]:
#Preparing product testing file
with open ("product_testfile.csv","w+") as product_testfile:
    fields_test_product=['MTV No.']+products
    writer_test_product = csv.DictWriter(product_testfile, fieldnames=fields_test_product)
    writer_test_product.writeheader()
    '''Read the data from excel'''
    FileName_test='Testing set.xlsx'
    df_test = pd.read_excel(FileName_test,sheet_name='Sheet1')
    ReadList_test=df_test.columns.values.tolist()
    #print(ReadList_test)
    #print(df_test['MTV Description'][0:])
    MTV_description_test_product=df_test[['MTV Description']][0:].values.tolist()
    #print(MTV_description_test[0])
    for i in range(0,len(MTV_description_test_product)):
        product_testfile.write(str(i+1))
        product_testfile.write('\t\t')
        #print(MTV_description[0])
        for desc3 in MTV_description_test_product[i]: 
            desc3_to_string = ''.join(desc3)
            desc3_to_string = desc3_to_string.lower()
            desc3_to_string=desc3_to_string.replace('.',' ')
            desc3_to_string=desc3_to_string.replace(',',' ')
            desc3_to_string=desc3_to_string.replace(':',' ')
            #print(desc1_to_string)
            desc3_words = desc3_to_string.split()
            #print(desc1_words) 
            for word3 in products:
                if word3 in desc3_words:
                    product_testfile.write('1\t\t')
                else:
                    product_testfile.write('0\t\t')
        product_testfile.write('\n')

In [61]:
product_featurefile_df = pd.read_csv('product_featurefile.csv',sep=',|\t\t|\n',header=0,engine='python')
product_featurefile_df.head()

,MTV No.,encompass,router,metro,e-wan,vpn,e-line,brains,ipvpn,vpn-dsl,swup,vpn-dsl.1,embed,shops.work/vpn/dsl,shops.work/unplg-s&m,shops.work/unplg-s&p,shops.work/unplg-so,l2/l3,Product
0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Encompass Router
1,2,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,Mtero e-wan vpn
2,3,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,Metro e-line vpn
3,4,1,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,BRAINS IPVPN
4,5,1,1,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,EMBED VIA VPN-DSL


In [62]:
product_testfile_df = pd.read_csv('product_testfile.csv',sep= ',|\t\t|\n',header=0,engine='python')
product_testfile_df.head()

,MTV No.,encompass,router,metro,e-wan,vpn,e-line,brains,ipvpn,vpn-dsl,swup,vpn-dsl.1,embed,shops.work/vpn/dsl,shops.work/unplg-s&m,shops.work/unplg-s&p,shops.work/unplg-so,l2/l3
0,1,1,1,1,1,1,0,1,1,1,1,1,1,0,0,0,0,1
1,2,1,1,1,1,1,0,1,1,1,1,1,1,0,0,0,0,1
2,3,1,1,1,1,1,0,1,1,1,1,1,1,0,0,0,0,1
3,4,1,1,1,1,1,0,1,1,1,1,1,1,0,0,0,0,1
4,5,1,1,1,1,1,0,1,1,1,1,1,1,0,0,0,0,1


In [63]:
product_featurefile_data = product_featurefile_df[products]
#featurefile_data = featurefile_df[['enterprise','customer','create','encompass','router','siebel','crm','odo','notification','provide','lead-to-cash','e-wan','vpn','new','creation','csr','e-line','vpn-dsl','dsl-vpn','unplg-s&m','cancellation','reason','clarity','service','order','partial','brains','ipvpn','full','cease','removing','deactivation']]
product_featurefile_data.head()

,encompass,router,metro,e-wan,vpn,e-line,brains,ipvpn,vpn-dsl,swup,vpn-dsl,embed,shops.work/vpn/dsl,shops.work/unplg-s&m,shops.work/unplg-s&p,shops.work/unplg-so,l2/l3
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0
4,1,1,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0


In [64]:
product_testfile_data = product_testfile_df[products]
#testfile_data = testfile_df[['enterprise','customer','create','encompass','router','siebel','crm','odo','notification','provide','lead-to-cash','e-wan','vpn','new','creation','csr','e-line','vpn-dsl','dsl-vpn','unplg-s&m','cancellation','reason','clarity','service','order','partial','brains','ipvpn','full','cease','removing','deactivation']]
product_testfile_data.head()

,encompass,router,metro,e-wan,vpn,e-line,brains,ipvpn,vpn-dsl,swup,vpn-dsl,embed,shops.work/vpn/dsl,shops.work/unplg-s&m,shops.work/unplg-s&p,shops.work/unplg-so,l2/l3
0,1,1,1,1,1,0,1,1,1,1,1,1,0,0,0,0,1
1,1,1,1,1,1,0,1,1,1,1,1,1,0,0,0,0,1
2,1,1,1,1,1,0,1,1,1,1,1,1,0,0,0,0,1
3,1,1,1,1,1,0,1,1,1,1,1,1,0,0,0,0,1
4,1,1,1,1,1,0,1,1,1,1,1,1,0,0,0,0,1


In [65]:
featurefile_target_product = product_featurefile_df['Product']
featurefile_target_product.head()

0     Encompass Router
1      Mtero e-wan vpn
2     Metro e-line vpn
3        BRAINS  IPVPN
4    EMBED VIA VPN-DSL
Name: Product, dtype: object

In [66]:
product_logreg = LogisticRegression()
feature_data_product_train,feature_data_product_test,feature_target_product_train,feature_target_product_test = train_test_split(product_featurefile_data,featurefile_target_product,test_size=0.3,random_state=3)

In [67]:
logreg.fit(feature_data_product_train,feature_target_product_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [68]:
target_pred_product = logreg.predict(feature_data_product_test)

In [69]:
product = logreg.predict(product_testfile_data)
print (product)

['L2/L3 VPN' 'L2/L3 VPN' 'L2/L3 VPN' 'L2/L3 VPN' 'L2/L3 VPN' 'L2/L3 VPN']


In [70]:
print(metrics.accuracy_score(feature_target_product_test,target_pred_product))

0.9333333333333333


In [71]:
def execute():
    for i in range(len(order_action)):
        var_1_obj = GetNextItem(order_action[i])
        var_1 = var_1_obj.get_next_item()
        while var_1 is not None:
            var_2_obj = GetNextItem(sales_channel[i])
            var_2 = var_2_obj.get_next_item()
            while var_2 is not None:
                var_3_obj = GetNextItem(product[i])
                var_3 = var_3_obj.get_next_item()
                while var_3 is not None:
                    print('Run a',var_1, 'flow for product', var_3, 'with sales channel', var_2)
                    var_3 = var_3_obj.get_next_item()               
                var_2 = var_2_obj.get_next_item()
            var_1 = var_1_obj.get_next_item()

In [72]:
class GetNextItem:

    def __init__(self, obj):
        self.obj = obj
        self.is_list = isinstance(obj, (list, tuple))
        self.is_exausted = False
        self.current_index = 0
    
    def get_next_item(self):
        if self.is_exausted:
            return None
        elif not self.is_list:
            self.is_exausted = True
            return self.obj
        else:
            try:
                val = self.obj[self.current_index]
            except IndexError:
                self.is_exausted = True
                val = None
            else:
                self.current_index += 1
            return val

In [73]:
execute()

Run a Provide flow for product L2/L3 VPN with sales channel Call centre
Run a Cease flow for product L2/L3 VPN with sales channel Call centre
Run a Cancel cease flow for product L2/L3 VPN with sales channel Call centre
Run a Cease flow for product L2/L3 VPN with sales channel Call centre
Run a Cancel Provide flow for product L2/L3 VPN with sales channel Call centre
Run a Cease flow for product L2/L3 VPN with sales channel Call centre
